In [1]:
import numpy as np
import string
from keras.callbacks import LambdaCallback
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Activation
from keras.models import Sequential

In [2]:
import pandas as pd
pd.__version__
df = pd.read_csv('C:/Users/LENOVO/Desktop/PCD/tweet.txt', sep="\n",header=None)
df

,0
0,"During our time in the White House, Michelle a..."
1,"There is a lot of disinformation out there, bu..."
2,Climate change is a real threat to us all—but ...
3,I’m proud to announce that the Obama President...
4,Black-owned independent bookstores play a crit...
...,...
61,"It might be Labor Day weekend, but let’s all r..."
62,Completing the Census is crucial because it wi...
63,Joe and Kamala have what it takes to lead this...
64,If you can volunteer as a poll worker for earl...


In [3]:
list_listword=[]#liste des listes des mots de chaque tweet
list_tweets=df[0].values.tolist()#liste des tweets
list_tweets

['During our time in the White House, Michelle and I were proud to display artwork from artists like Alma Thomas and Charles Alston—whose work provoked thought, challenged our assumptions, and shaped how we define our narrative as a country.',
 "There is a lot of disinformation out there, but here’s the truth: You should get a COVID vaccine as soon as it's available to you. It could save your life—or a loved one’s.",
 'Climate change is a real threat to us all—but feeling helpless or overwhelmed isn’t going to solve it. As long as we stay hopeful and keep working with the urgency the challenge demands, we can make a difference.',
 'I’m proud to announce that the Obama Presidential Center will officially break ground in 2021. Our hope is that the center will breathe new life into historic Jackson Park while delivering jobs, growth, and much more to the South Side. Let’s get to work.',
 'Black-owned independent bookstores play a critical role in communities all across our country. I want

In [4]:
import re
l=[]
for text in list_tweets:
    text = re.sub(r'http\S+', '', text)   # Remove URLs
    text = re.sub(r'—', ' ', text) 
    text = re.sub(r'@[a-zA-Z0-9_]+', '', text)  # Remove @ mentions
    text = text.strip(" ")   # Remove whitespace resulting from above
    text = re.sub(r' +', ' ', text)   # Remove redundant spaces
    l.append(text)
list_tweets=l
list_tweets

['During our time in the White House, Michelle and I were proud to display artwork from artists like Alma Thomas and Charles Alston whose work provoked thought, challenged our assumptions, and shaped how we define our narrative as a country.',
 "There is a lot of disinformation out there, but here’s the truth: You should get a COVID vaccine as soon as it's available to you. It could save your life or a loved one’s.",
 'Climate change is a real threat to us all but feeling helpless or overwhelmed isn’t going to solve it. As long as we stay hopeful and keep working with the urgency the challenge demands, we can make a difference.',
 'I’m proud to announce that the Obama Presidential Center will officially break ground in 2021. Our hope is that the center will breathe new life into historic Jackson Park while delivering jobs, growth, and much more to the South Side. Let’s get to work.',
 'Black-owned independent bookstores play a critical role in communities all across our country. I want

In [5]:
max_sentence_len = 200
#sentences=[]
#for i in list_tweets:
   #sentences.append(i.split()[:max_sentence_len])
sentences = [[word for word in doc.lower().translate(string.punctuation).split()[:max_sentence_len]] for doc in list_tweets]
sentences

[['during',
  'our',
  'time',
  'in',
  'the',
  'white',
  'house,',
  'michelle',
  'and',
  'i',
  'were',
  'proud',
  'to',
  'display',
  'artwork',
  'from',
  'artists',
  'like',
  'alma',
  'thomas',
  'and',
  'charles',
  'alston',
  'whose',
  'work',
  'provoked',
  'thought,',
  'challenged',
  'our',
  'assumptions,',
  'and',
  'shaped',
  'how',
  'we',
  'define',
  'our',
  'narrative',
  'as',
  'a',
  'country.'],
 ['there',
  'is',
  'a',
  'lot',
  'of',
  'disinformation',
  'out',
  'there,',
  'but',
  'here’s',
  'the',
  'truth:',
  'you',
  'should',
  'get',
  'a',
  'covid',
  'vaccine',
  'as',
  'soon',
  'as',
  "it's",
  'available',
  'to',
  'you.',
  'it',
  'could',
  'save',
  'your',
  'life',
  'or',
  'a',
  'loved',
  'one’s.'],
 ['climate',
  'change',
  'is',
  'a',
  'real',
  'threat',
  'to',
  'us',
  'all',
  'but',
  'feeling',
  'helpless',
  'or',
  'overwhelmed',
  'isn’t',
  'going',
  'to',
  'solve',
  'it.',
  'as',
  'long',

In [6]:
import gensim
print('\nTraining word2vec...')
word_model = gensim.models.Word2Vec(sentences, size=100, min_count=1, window=5, iter=100)
pretrained_weights = word_model.wv.vectors
vocab_size, emdedding_size = pretrained_weights.shape
print('Result embedding shape:', pretrained_weights.shape)
print('Checking similar words:')
for word in sentences[0]:
  most_similar = ', '.join('%s (%.2f)' % (similar, dist) for similar, dist in word_model.wv.most_similar(word)[:8])
  print('  %s -> %s' % (word, most_similar))


Training word2vec...
Result embedding shape: (957, 100)
Checking similar words:
  during -> years (1.00), first (1.00), presidency. (1.00), shaped (1.00), send (1.00), played (1.00), across (1.00), try (1.00)
  our -> communities (1.00), democracy. (1.00), country. (1.00), shores. (1.00), everything (1.00), decade. (1.00), of (1.00), reminded (1.00)
  time -> mail, (1.00), touch (1.00), share (1.00), now (1.00), cast (0.99), decided (0.99), them. (0.99), boldness (0.99)
  in -> runoff (1.00), minutes (1.00), this (1.00), few (1.00), time. (1.00), i'll (1.00), joel, (1.00), take (1.00)
  the -> latest (1.00), just (1.00), ever (1.00), examples (1.00), are (1.00), coast (1.00), obama (1.00), very (1.00)
  white -> house (1.00), literally (1.00), playbook (1.00), would (1.00), were (1.00), tim (1.00), role (1.00), administration (1.00)
  house, -> white (1.00), were (1.00), michelle (1.00), land (1.00), display (1.00), playlist, (1.00), proud (1.00), figured (1.00)
  michelle -> met. (1.

In [7]:
def word2idx(word):
  return word_model.wv.vocab[word].index
def idx2word(idx):
  return word_model.wv.index2word[idx]

In [8]:
print('\nPreparing the data for LSTM...')
train_x = np.zeros([len(sentences), max_sentence_len], dtype=np.int32)
train_y = np.zeros([len(sentences)], dtype=np.int32)
for i, sentence in enumerate(sentences):
  for t, word in enumerate(sentence[:-1]):
    train_x[i, t] = word2idx(word)
  train_y[i] = word2idx(sentence[-1])
print('train_x shape:', train_x.shape)
print('train_y shape:', train_y.shape)


Preparing the data for LSTM...
train_x shape: (66, 200)
train_y shape: (66,)


In [9]:
print('\nTraining LSTM...')
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[pretrained_weights]))
model.add(LSTM(units=emdedding_size))
model.add(Dense(units=vocab_size))
model.add(Activation('softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')


Training LSTM...


In [10]:
def sample(preds, temperature=1.0):
  if temperature <= 0:
    return np.argmax(preds)
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [11]:
def generate_next(text, num_generated=10):
  word_idxs = [word2idx(word) for word in text.lower().split()]
  for i in range(num_generated):
    prediction = model.predict(x=np.array(word_idxs))
    idx = sample(prediction[-1], temperature=0.7)
    word_idxs.append(idx)
  return ' '.join(idx2word(idx) for idx in word_idxs)

In [12]:
def on_epoch_end(epoch, _):
  print('\nGenerating text after epoch: %d' % epoch)
  texts = sentences[0]
  for text in texts:
    sample = generate_next(text)
    print('%s... -> %s' % (text, sample))

model.fit(train_x, train_y,
          batch_size=128,
          epochs=10,
          callbacks=[LambdaCallback(on_epoch_end=on_epoch_end)])

Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 6.8756
Generating text after epoch: 0
during... -> during now, you’re lady, wear lives, format. that's smart send states,
our... -> our things. if poll in eviction played coming worker lot, presidency.
time... -> time country. dr. vote: early. start, protections most may nation, continue
in... -> in masks save dylan robinson. violence, land, us expanded hotline determination
the... -> the me. three stay day. care, tim determination your answer unimaginable
white... -> white recently. hard can’t lead south home. volunteer support strongest listen
house,... -> house, look control, sharing celebrating covid-19 nation, i'll seen smart voices
michelle... -> michelle show spend place, examples complacent. read baseball americans show than
and... -> and this reason let's charles three lynchings, wanted traitorous right promised
i... -> i book brothas alston tribute last here's i’m does. cherish ever
were... -> were smiling goin

michelle... -> michelle wearing difference supreme black; well, obviously, worker testament you’re taking
and... -> and and actions through not few enter ever, communities. virtually against
i... -> i not holidays me. yourself wait that's lights partner, marched. mind,
were... -> were 3: they holidays assumptions, pandemic years happy candidates––and reaching traitorous
proud... -> proud provide helping police define working world volunteer levels. from monica,
to... -> to beings. events lynchings, obligation: memoir, strength, bring hank continue other
display... -> display hank robinson provoked officially tool 2021. say put decision. right
artwork... -> artwork was millions actors helpless will played new. going finally here's
from... -> from planning if facetime help 11 let's individual, home hours decided
artists... -> artists joe boldness reached officially lights early revisited helpless do, labor
like... -> like regardless call gave actors heard, there, actors office. years, ly

artwork... -> artwork hope. as failed care even heard towering isn’t boldness mail-in-ballot
from... -> from blessing. drop elections leader day. until we’re answer getting ballot.
artists... -> artists democracy. choices moment unconscionable listening actions turn artwork i’m overwhelmed
like... -> like even room he folks hotline unlike critical smith, surprise widely
alma... -> alma anyway polls soon anyone including early. found volunteer very families
thomas... -> thomas getting voice. then can’t an stake, at just week, time.
and... -> and independent strength, check-in pandemic life listening well, well fun. better.
charles... -> charles better send all-time delivering. learned across vaccinated senate christmas ahead.
alston... -> alston line reach lead cases over doing, bookstores mail, helping baton
whose... -> whose never young, one, andrés slurs, tests, black-owned aligns under traitorous
work... -> work weekend, years. sharing sharing communities. smith, willingness obligat

charles... -> charles save park communities. days the too. walks do staying independent
alston... -> alston conditions that’s week, snapshot already far smart pain family. bob
whose... -> whose remake made it. plan know off blurring safe thing injury
work... -> work inspires declared start, life-threatening control, reminder matter ticket, protections. use
provoked... -> provoked dylan delivering vote, playbook stands many ringing much funding need
thought,... -> thought, reminder complacent. why do, were protections. volunteer here’s coming just
challenged... -> challenged touch from playlist, pandemic protections battle millions helping outstanding you
our... -> our others power honest same. put better. reason waved as more
assumptions,... -> assumptions, aimed down americans down snapshot election. up look. hank protect
and... -> and most only towering hope coast partner, alma lewis elections reason
shaped... -> shaped surely heart off years. lead take music vote. lead work.
how... 

our... -> our tribute other smart lights powerful now. dylan jobs, presidency. blurring
assumptions,... -> assumptions, conditions minutes pain theatrical that clearly the if she stuck
and... -> and speedy threat endorse quarterback box bob stake offer events extending
shaped... -> shaped stake, threaten shown sameach! direction three provoked available, talked hotline.
how... -> how social up may paris give dropping heroes candidates––and that's quarterback
we... -> we lot midst thank good battle beings. widely early have tim
define... -> define everyone voting right, able counting more aaron black-owned obviously, democratic
our... -> our michelle protect let’s brothas list hope. stole hotline. u.s. wearing
narrative... -> narrative them leader facetime left. shown care, pitches friend, coast optimism
as... -> as ground stakes look. blessing. whenever friend, plan mask. enjoyed faced
a... -> a honor run pm, honest republicans america completing widely kids between
country.... -> coun